#  Inférence avec un modèle CamemBERT pour la classification de texte pour froid file

In [6]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from transformers import CamembertTokenizer, CamembertForSequenceClassification

# Charger le modèle et le tokenizer
model = CamembertForSequenceClassification.from_pretrained("./saved_model")
tokenizer = CamembertTokenizer.from_pretrained("./saved_model")
model.eval()

# Chargement des données de test
df = pd.read_csv('./data/froid_new.csv').dropna()
df=df[:500]
# Encodage des labels
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["CLASSE"])

# Préparation des textes et des labels
texts = df['TEXTE_AUTRE'].tolist()
true_labels = df['CLASSE'].tolist()

# Tokenisation
encodings = tokenizer(
    texts,
    padding="max_length",
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Détection de l'appareil
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Inférence
with torch.no_grad():
    outputs = model(
        input_ids=encodings["input_ids"].to(device),
        attention_mask=encodings["attention_mask"].to(device)
    )
    predictions = torch.argmax(outputs.logits, dim=-1)

# Décodage des prédictions
predicted_labels = label_encoder.inverse_transform(predictions.cpu().numpy())

# Affichage des résultats
correct = sum(p == t for p, t in zip(predicted_labels, true_labels))
for text, pred, true in zip(texts, predicted_labels, true_labels):
    print(f"> {text}\n  → Prédit : {pred}\n  → Réel : {true}\n")

print(f"Précision : {correct / len(texts):.4f}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


> problème application mobile
  → Prédit : Application Mobile/Ch.net
  → Réel : Application Mobile/Ch.net

> mieux communiquer avec le client
  → Prédit : Service en agence
  → Réel : Service en agence

> amélioration sur la qualité des personnelles
  → Prédit : Service en agence
  → Réel : Service en agence

> être  en phase avec les banque concurrent ,problème des messages n'est pas à jour
  → Prédit : Service en agence
  → Réel : Service en agence

> problème des messages n'est pas à jour ,mieux communiquer avec le client
  → Prédit : Service en agence
  → Réel : Service en agence

> retard des notifications dans l'application - l'application pocket bank n'est pas à jour -
  → Prédit : Application Mobile/Ch.net
  → Réel : Application Mobile/Ch.net

> manque des agents
  → Prédit : Attente en agence
  → Réel : Attente en agence

> souci de transfére de compte
  → Prédit : Service en agence
  → Réel : Service en agence

> le client réclame que ses demandes ne sont pas traités au nivea

# Pour une phrase

In [5]:
# Phrase à prédire
text = "gab pas de solde en weekend ou bien en maintenance"

# Prétraitement
encoding = tokenizer(
    text,
    padding="max_length",
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

with torch.no_grad():
    outputs = model(
        input_ids=encoding["input_ids"].to(device),
        attention_mask=encoding["attention_mask"].to(device)
    )
    predicted_class = torch.argmax(outputs.logits, dim=-1)

# Décodage du label prédict
predicted_label = label_encoder.inverse_transform(predicted_class.cpu().numpy())[0]

print(f"Texte : {text}")
print(f"Classe prédite : {predicted_label}")

Texte : gab pas de solde en weekend ou bien en maintenance
Classe prédite : Problèmes Gab
